In [ ]:
import sys
!{sys.executable} -m pip install --upgrade tensorflow  

In [55]:
import torch
import pandas as pd
import numpy as np

df = pd.read_csv('./../Naive_Bayes/tweets/allLabeledTweets.csv')
df.head()

,message,id,tweetId,createdAt,language,inReplyToStatusId,inReplyToUserId,inReplyToScreenName,userId,userName,...,retweetedId,retweetCount,label,message_lowercase,clean_message,clean_message_no_punct,clean_message_no_stopwords_from_list,clean_message_no_punct_no_stopwords_from_list,clean_message_no_punct_no_freq_stopwords,clean_message_no_freq_stopwords
0,"Rodas sajūta, ka arī @airBaltic ir kaut kāds @...",0,1213615462581440500,2020-01-05T00:17:28+00:00,lv,NaN,NaN,NaN,62004316,Edgars Eglītis,...,NaN,0,2,"rodas sajūta, ka arī @airbaltic ir kaut kāds @...","rodas sajūta, ka arī MENTION ir kaut kāds MENT...",rodas sajūta ka arī MENTION ir kaut kāds MENT...,"rodas sajūta , MENTION kāds MENTION kaktu kant...",rodas sajūta MENTION kāds MENTION kaktu kantor...,rodas sajūta MENTION kaut MENTION kaktu kantor...,"rodas sajūta , MENTION kaut MENTION kaktu kant..."
1,"Amsterdama, @airBaltic smukulītes 👍🏻 https://...",0,1213740889476128800,2020-01-05T08:35:52+00:00,lv,NaN,NaN,NaN,213752948,Dairis Kuciks,...,NaN,0,1,"amsterdama, @airbaltic smukulītes 👍🏻 https://...","amsterdama, MENTION smukulītes 👍🏻 URL",amsterdama MENTION smukulītes 👍🏻 URL,"amsterdama , MENTION smukulītes 👍🏻 URL",amsterdama MENTION smukulītes 👍🏻 URL,amsterdama MENTION smukulītes 👍🏻 URL,"amsterdama , MENTION smukulītes 👍🏻 URL"
2,Kārtējo reizi... \r\n@airBaltic vakar raudošai...,0,1214174186069012500,2020-01-06T13:17:38+00:00,lv,NaN,NaN,NaN,1107743410646069200,Selma,...,NaN,73,2,kārtējo reizi... \r\n@airbaltic vakar raudošai...,kārtējo reizi... MENTION vakar raudošai sievie...,kārtējo reizi MENTION vakar raudošai sievie...,kārtējo reizi ... MENTION vakar raudošai sievi...,kārtējo reizi MENTION vakar raudošai sievietei...,kārtējo reizi MENTION vakar raudošai sievietei...,kārtējo reizi ... MENTION vakar raudošai sievi...
3,80.05% airBaltic akciju pieder Latvijas valsti...,0,1214176732456075300,2020-01-06T13:27:45+00:00,lv,NaN,NaN,NaN,1107743410646069200,Selma,...,NaN,2,2,80.05% airbaltic akciju pieder latvijas valsti...,NMBR % airbaltic akciju pieder latvijas valsti...,NMBR airbaltic akciju pieder latvijas valsti...,NMBR % airbaltic akciju pieder latvijas valsti...,NMBR airbaltic akciju pieder latvijas valstij ...,NMBR airbaltic akciju pieder latvijas valstij ...,NMBR % airbaltic akciju pieder latvijas valsti...
4,Laba ideja lidojumiem ar @airBaltic https://t....,0,1214197047382941700,2020-01-06T14:48:29+00:00,lv,NaN,NaN,NaN,110783755,Inga Gorbunova,...,NaN,4,0,laba ideja lidojumiem ar @airbaltic https://t....,laba ideja lidojumiem ar MENTION URL,laba ideja lidojumiem ar MENTION URL,laba ideja lidojumiem MENTION URL,laba ideja lidojumiem MENTION URL,laba ideja lidojumiem MENTION URL,laba ideja lidojumiem MENTION URL


In [56]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['label', 'data_type']).count()

message   id  tweetId  createdAt  language  \
label data_type                                               
0     train          205  205      205        205       205   
      val             37   37       37         37        37   
1     train           71   71       71         71        71   
      val             13   13       13         13        13   
2     train          103  103      103        103       103   
      val             18   18       18         18        18   

                 inReplyToStatusId  inReplyToUserId  inReplyToScreenName  \
label data_type                                                            
0     train                      0                0                    0   
      val                        0                0                    0   
1     train                      0                0                    0   
      val                        0                0                    0   
2     train                      0                0                    0   
      val                        0                0                    0   

                 userId  userName  ...  placeType  retweetedId  retweetCount  \
label data_type                    ...                                         
0     train         205       205  ...          9            0           205   
      val            37        37  ...          0            0            37   
1     train          71        71  ...          5            0            71   
      val            13        13  ...          4            0            13   
2     train         103       103  ...         10            0           103   
      val            18        18  ...          0            0            18   

                 message_lowercase  clean_message  clean_message_no_punct  \
label data_type                                                             
0     train                    205            205                     205   
      val                       37             37                      37   
1     train                     71             71                      71   
      val                       13             13                      13   
2     train                    103            103                     103   
      val                       18             18                      18   

                 clean_message_no_stopwords_from_list  \
label data_type                                         
0     train                                       205   
      val                                          37   
1     train                                        71   
      val                                          13   
2     train                                       103   
      val                                          18   

                 clean_message_no_punct_no_stopwords_from_list  \
label data_type                                                  
0     train                                                205   
      val                                                   37   
1     train                                                 71   
      val                                                   13   
2     train                                                103   
      val                                                   18   

                 clean_message_no_punct_no_freq_stopwords  \
label data_type                                             
0     train                                           205   
      val                                              37   
1     train                                            71   
      val                                              13   
2     train                                           103   
      val                                              18   

                 clean_message_no_freq_stopwords  
label data_type                                   
0     train                                  205  
      val        

In [57]:
tokenizer = BertTokenizer.from_pretrained('./models6/lvbert_pytorch/')

In [59]:
from torch.utils.data import TensorDataset

encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].message_lowercase.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].message_lowercase.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [60]:
# from transformers import BertTokenizer

# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# example_text = 'I will watch Memento tonight'
# bert_input = tokenizer(example_text,padding='max_length', max_length = 10, 
#                        truncation=True, return_tensors="pt")


# print(bert_input['input_ids'])
# print(bert_input['token_type_ids'])
# print(bert_input['attention_mask'])

In [61]:
# example_text = tokenizer.decode(bert_input.input_ids[0])

# print(example_text)

In [26]:
# class Dataset(torch.utils.data.Dataset):

#     def __init__(self, df):

#         self.labels = [label for label in df['label']]
#         self.texts = [tokenizer(text, 
#                                padding='max_length', max_length = 512, truncation=True,
#                                 return_tensors="pt") for text in df['message']]

#     def classes(self):
#         return self.labels

#     def __len__(self):
#         return len(self.labels)

#     def get_batch_labels(self, idx):
#         # Fetch a batch of labels
#         return np.array(self.labels[idx])

#     def get_batch_texts(self, idx):
#         # Fetch a batch of inputs
#         return self.texts[idx]

#     def __getitem__(self, idx):

#         batch_texts = self.get_batch_texts(idx)
#         batch_y = self.get_batch_labels(idx)

#         return batch_texts, batch_y

In [27]:
# from torch.optim import Adam
# from tqdm import tqdm

# def train(model, train_data, val_data, learning_rate, epochs):

#     train, val = Dataset(train_data), Dataset(val_data)

#     train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
#     val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

#     use_cuda = torch.cuda.is_available()
#     device = torch.device("cuda" if use_cuda else "cpu")

#     criterion = nn.CrossEntropyLoss()
#     optimizer = Adam(model.parameters(), lr= learning_rate)

#     if use_cuda:

#             model = model.cuda()
#             criterion = criterion.cuda()

#     for epoch_num in range(epochs):

#             total_acc_train = 0
#             total_loss_train = 0

#             for train_input, train_label in tqdm(train_dataloader):

#                 train_label = train_label.to(device)
#                 mask = train_input['attention_mask'].to(device)
#                 input_id = train_input['input_ids'].squeeze(1).to(device)

#                 output = model(input_id, mask)
                
#                 batch_loss = criterion(output, train_label)
#                 total_loss_train += batch_loss.item()
                
#                 acc = (output.argmax(dim=1) == train_label).sum().item()
#                 total_acc_train += acc

#                 model.zero_grad()
#                 batch_loss.backward()
#                 optimizer.step()
            
#             total_acc_val = 0
#             total_loss_val = 0

#             with torch.no_grad():

#                 for val_input, val_label in val_dataloader:

#                     val_label = val_label.to(device)
#                     mask = val_input['attention_mask'].to(device)
#                     input_id = val_input['input_ids'].squeeze(1).to(device)

#                     output = model(input_id, mask)

#                     batch_loss = criterion(output, val_label)
#                     total_loss_val += batch_loss.item()
                    
#                     acc = (output.argmax(dim=1) == val_label).sum().item()
#                     total_acc_val += acc
            
#             print(
#                 f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
#                 | Train Accuracy: {total_acc_train / len(train_data): .3f} \
#                 | Val Loss: {total_loss_val / len(val_data): .3f} \
#                 | Val Accuracy: {total_acc_val / len(val_data): .3f}')

In [41]:
# from transformers import BertForSequenceClassification

# # model = torch.load('./models6/lvbert_pytorch/pytorch_model.bin')
# # model = BertForSequenceClassification.from_pretrained('./models6/lvbert_pytorch/pytorch_model.bin')
# model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased",
#                                                       num_labels=3,
#                                                       output_attentions=False,
#                                                       output_hidden_states=False)

# # model.to(torch.device('cpu'))

In [42]:
# model.load_state_dict(torch.load('./models6/lvbert_pytorch/pytorch_model.bin', map_location=torch.device('cpu')))
# EPOCHS = 5
# LR = 1e-6
              
# train(model, df_train, df_val, LR, EPOCHS)

AttributeError: 'collections.OrderedDict' object has no attribute 'load_state_dict'

In [53]:
# tokenizer = BertTokenizer.from_pretrained('./models6/lvbert_pytorch/')

# # Copy the model to the GPU.
# model.to(torch.device('cpu'))

Some weights of the model checkpoint at ./models6/lvbert_pytorch/ were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model check

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32004, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [62]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [70]:
model = BertForSequenceClassification.from_pretrained('./models6/lvbert_pytorch/',num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at ./models6/lvbert_pytorch/ were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model check

In [71]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [72]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict = {0: 0, 1: 1, 2: 2,}
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [73]:
import random
import numpy as np

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(torch.device('cpu')) for b in batch)
        
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [89]:
model.to(torch.device('cpu'))
from torch import nn
criterion = nn.CrossEntropyLoss()

for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False, position=0, colour='green', ncols=80)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(torch.device('cpu')) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
        
    torch.save(model.state_dict(), f'models6/finetuned_lvBERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|                                                                                            | 0/5 [12:20<?, ?it/s]


Epoch 1
Training loss: 0.7400127831171817


 20%|████████████████▌                                                                  | 1/5 [13:16<53:06, 796.62s/it]

Validation loss: 0.7208103641219761
F1 Score (Weighted): 0.7227562536642563


 20%|████████████████▌                                                                  | 1/5 [25:32<53:06, 796.62s/it]

...


Epoch 3
Training loss: 0.7480769972866914


 60%|█████████████████████████████████████████████████▊                                 | 3/5 [40:33<27:14, 817.02s/it]

Validation loss: 0.7208103641219761
F1 Score (Weighted): 0.7227562536642563


 60%|█████████████████████████████████████████████████▊                                 | 3/5 [40:52<27:14, 817.45s/it]


KeyboardInterrupt: 